In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/hc-11th/sub_f8.csv
/kaggle/input/hc-12th/sub_f8.csv
/kaggle/input/santa-2023/sample_submission.csv
/kaggle/input/santa-2023/puzzles.csv
/kaggle/input/santa-2023/puzzle_info.csv


In [3]:
from tqdm import tqdm
import time
from datetime import timedelta
import igraph as ig
import psutil

In [4]:
puzzle_info_df = pd.read_csv('/kaggle/input/santa-2023/puzzle_info.csv')
#puzzle_info_df.head()
puzzles_df = pd.read_csv('/kaggle/input/santa-2023/puzzles.csv')
puzzles_df

allowed_moves = {}

for _, row in puzzle_info_df.iterrows():
    allowed_moves[row['puzzle_type']] = eval(row['allowed_moves'])

In [5]:
#submission_df = pd.read_csv('/kaggle/input/first-hc/submission_5_334.csv')
submission_df = pd.read_csv('/kaggle/input/hc-12th/sub_f8.csv')
submission_df

,id,moves
0,0,r1.-f1
1,1,f0.r1.f1.-d0.-d0.-f0.-r0.f0.d0
2,2,-d1.-r0.f0.-r1.f1.d1.-r1.-f0.d1.f0.d1.d1
3,3,-f0.d0.-r0.f0.-d0.-r0.d0.-f0.-r0.-f0
4,4,-r1.-f0.d0.r0.-d1.-d1.r1.d1.f0.r1.-d1.-r1
...,...,...
393,393,f19.f21.-f39.f20.f2.-f5.f7.-r3.f55.-f12.f65.-f...
394,394,-f31.-f22.f16.-f17.-f13.-f24.-f14.f2.f21.f44.f...
395,395,-r0.-f42.-f8.f16.-f49.f14.-f1.f56.f26.f35.f62....
396,396,f25.-f29.f46.f49.-f8.f27.f26.-f20.f2.-f20.f6.f...


In [6]:
submission_df['moves'].str.split('.').apply(lambda x: len(x)).sum()

776643

In [7]:
def move_state(state, move, moves_pool):
    
    if '-' in move:
        move = move[1:]
        rev = True
    else:
        rev = False

    move = moves_pool[move]

    if rev:
        new_state = state[:]
        for i in range(len(move)):
            new_state[move[i]] = state[i]
        state = new_state
    else:
        state = [state[idx] for idx in move]
    
    return state

def revMove(move):
    if '-' in move: return move[1:]
    else: return '-' + move

def show_puzzle_info(puzzle_id):
    current_puzzle = puzzles_df.loc[puzzle_id]
    puzzle_type = current_puzzle['puzzle_type']
    initial_state = current_puzzle['initial_state']
    solution_state = current_puzzle['solution_state']
    
    moves = submission_df.loc[puzzle_id]['moves'].split('.')
    moves_pool = allowed_moves[puzzle_type]
    print('puzzle_id', puzzle_id, '       puzzle_type', puzzle_type)
    num_wildcards = current_puzzle['num_wildcards']
    if num_wildcards != 0: print('****** WILDCARDS **', num_wildcards, 'number of move types', len(moves_pool))
    else: print('number of move types', len(moves_pool))
    print('number of moves in sample solution', len(moves))
    str_solution_state = solution_state.split(';')
    nunique_colors = pd.Series(str_solution_state).nunique()
    print('nunique_colors', nunique_colors)
    #print(moves_pool)
    #first_move_type = moves_pool.values()
    #print('first_move_type', first_move_type)
    print('number of faces', len(str_solution_state))
    #print(solution_state)


def solve_random_puzzle_segment(puzzle_id, maxNodes =  4000000, maxEdges = 11000000, segment_length = 250):
    V = True
    #segment_length = 250
    ts = time.time()
    current_puzzle = puzzles_df.loc[puzzle_id]
    puzzle_type = current_puzzle['puzzle_type']
    initial_state = current_puzzle['initial_state']
    solution_state = current_puzzle['solution_state']
    num_wildcards = current_puzzle['num_wildcards']
    
    moves = submission_df.loc[puzzle_id]['moves'].split('.')
    if (len(moves) < 100) or (len(moves) < 2 * segment_length): 
        print('oooooooooooooooooooooooooooooooooooooooooooooooooooooooo')
        print('skipping this puzzle becaue so few moves', len(moves))
        print('maybe you should use solve_entire_puzzle() ')
        print('oooooooooooooooooooooooooooooooooooooooooooooooooooooooo')
        return(submission_df.loc[puzzle_id]['moves'])
    moves_pool = allowed_moves[puzzle_type]
    #if num_wildcards != 0: print('****** WILDCARDS **', num_wildcards)
    
    state = initial_state.split(';')
    
    state_list = []
    state_list.append(state)
    
    for move in moves:
        state = move_state(state, move, moves_pool)
        state_list.append(state)
        
    print(' state_list[] traced out. Elapsed', timedelta(seconds = (time.time() - ts)))
    # now we need to pick a random subsegment and pass that into solve_segment
    start_pt = np.random.randint(len(state_list) - (segment_length+1))
    end_pt = start_pt + segment_length
    if V: print('start, end',start_pt, end_pt)
    
    before_moves = moves[:start_pt]
    seg_moves = moves[start_pt:(end_pt+0)]
    after_moves = moves[(end_pt + 0):]
    if len(before_moves) + len(seg_moves) + len(after_moves) != len(moves):
        print('I cannot do math')
        return(submission_df.loc[puzzle_id]['moves'])
                                 
    seg_initial_state, seg_solution_state = state_list[start_pt], state_list[end_pt]
    seg_initial_state = ';'.join(seg_initial_state)
    seg_solution_state = ';'.join(seg_solution_state)
    print('clearing state_list[]')
    state_list.clear()
    # ---------------------------------
    res = solve_segment(puzzle_id, seg_moves, moves_pool, seg_initial_state, seg_solution_state, 0, maxNodes, maxEdges) # ignore wild cards
    # ---------------------------------
    
    if res == 'no joy!': return(submission_df.loc[puzzle_id]['moves'])
    # spice the moves together and test
    total_moves = moves[:start_pt]
    total_moves.extend(res.split('.'))
    total_moves.extend(moves[end_pt:])
    res = total_moves
    # **********************************
    # double check
    state = initial_state.split(';')
    count = 0
    for move in res:
        state = move_state(state, move, moves_pool)
        #print(count, move, ' state:', state)
        count+= 1
    
    # Check that submitted moves solve puzzle
    #num_wrong_facelets = beauty(solution_state, state)
    num_wrong_facelets = sum(not(s == t) for s, t in zip(solution_state, ';'.join(state)))
    #if V:
    #    print('ending state:\n', ';'.join(state))
    #    print('\nsolution_state:\n', solution_state)
    print('  spliced    count', count, '                  num_wrong_facelets', num_wrong_facelets)
    if num_wrong_facelets > num_wildcards:
        print('   *********************** UH OH ****************')
        print('   *********************** UH OH ****************')
        print('   *********************** UH OH ****************')
        print('   *********************** UH OH ****************')
        return(submission_df.loc[puzzle_id]['moves'])
    return('.'.join(res))

def solve_entire_puzzle(puzzle_id, maxNodes =  4000000, maxEdges = 11000000):
    V = False
    ts = time.time()
    current_puzzle = puzzles_df.loc[puzzle_id]
    puzzle_type = current_puzzle['puzzle_type']
    initial_state = current_puzzle['initial_state']
    solution_state = current_puzzle['solution_state']
    num_wildcards = current_puzzle['num_wildcards']
    
    moves = submission_df.loc[puzzle_id]['moves'].split('.')
    if len(moves) > 6000: 
        print('oooooooooooooooooooooooooooooooooooooooooooooooooooooooo')
        print('skipping this puzzle becaue too many moves', len(moves))
        print('oooooooooooooooooooooooooooooooooooooooooooooooooooooooo')
        return(submission_df.loc[puzzle_id]['moves'])
    moves_pool = allowed_moves[puzzle_type]
    if num_wildcards != 0: print('****** WILDCARDS **', num_wildcards)
    
    state = initial_state.split(';')
    
    state_list = []
    state_list.append(state)
    
    for move in moves:
        state = move_state(state, move, moves_pool)
        state_list.append(state)
        
    res = solve_segment(puzzle_id, moves, moves_pool, initial_state, solution_state, num_wildcards, maxNodes, maxEdges)
    
    if res == 'no joy!': return(submission_df.loc[puzzle_id]['moves'])
        
    return(res)
        
""" Test to avoid exceeding memory for the notebook """
def check_memory():
    # Get the used memory in bytes
    used_memory = psutil.virtual_memory().used
    # Convert it to gigabytes
    used_memory_gb = used_memory / (1024 ** 3)
    # Set the threshold to 28 GB
    threshold = 25
    # Compare the used memory with the threshold
    if used_memory_gb > threshold:
        # Return a warning message
        print("Memory is getting high: Your Jupyter notebook is using", used_memory_gb, "GB of memory, which exceeds the safety limit of", threshold, " GB.")
        print('We will wrap up work on this puzzle')
        return(True)
    else:
        # Return a normal message
        #print (f"Your Jupyter notebook is using {used_memory_gb:.2f} GB of memory, which is within the limit of {threshold} GB.")
        return(False)


""" 
Take a valid path (moves), 
find the edges, 
expand the nodes, 
expand the neighbors, etc. 
Place the edges into an array
Call igraph to get the shortest path
Use undirected graph
"""
def solve_segment(puzzle_id, moves, moves_pool, initialState, solutionState, num_wildcards, maxNodes =  4000000, maxEdges = 11000000):
    V = False
    ts = time.time()
    current_puzzle = puzzles_df.loc[puzzle_id]
    puzzle_type = current_puzzle['puzzle_type']
    
    #G = nx.DiGraph()
    #G.add_nodes_from(range(maxNodes))
    #listNodes = [] # list( range(maxNodes))
    stateD = {} # holding the nodes in a Dictionary is MUCH faster than a list for large graphs
    edgesD = {} # holds moves for each forward edge (and the keys hold the edges)
    edgesR = {} # holds moves for each reverse edge (Might not be worth the extra memory)
    newLeaves = np.empty( maxNodes, dtype = object)
    
    state = initialState.split(';')

    if initialState == solutionState:
        print(" initial state and solution state are same.")
    # trace initial path 
    n = 0 # number of nodes
    #listNodes.append(initialState)#[n] = initialState #.append(postState)
    stateD[initialState] = n
    newLeaves[n] = initialState #..append(postState)
    n += 1
    preState = initialState
    for move in moves:
        state = move_state(state, move, moves_pool)
        postState = ";".join(state)
        if postState not in stateD:
            #listNodes.append(postState)#[n] = postState #.append(postState)
            stateD[postState] = n
            newLeaves[n] = postState #..append(postState)
            n += 1
        idPre  = stateD[ preState] # listNodes.index(preState)
        idPost = stateD[postState] # listNodes.index(postState)
        #G.add_edge(idPre, idPost, weight=1.0, move=f"{move}")
        #G.add_edge(idPost, idPre, weight=1.0, move=f"{revMove(move)}")
        edgesD[idPre, idPost] = move
        edgesR[idPost, idPre] = revMove(move)
        preState = postState
    leafNodes = np.copy (newLeaves[:n])
    if solutionState != postState:
        print('solutionState != postState. maybe because there are wildcards.....')
        solutionState = postState   #
    if (initialState not in stateD) or (solutionState not in stateD):
        print('\n\n Problem \n\n\n')
        print('(initialState not in stateD) or (solutionState not in stateD). no joy')
        return('no joy!')
    # We have traced the initial solution into a graph. Now we start expanding to look at neighboring nodes    
    Done = False
    times_expanded = 0
    memory_warning = check_memory()
    while not Done:
        k = 0
        # do one simple expand
        if V: print('   nodes, leaves, edges', n, len(leafNodes), len(edgesD))
        times_expanded += 1
        # tqdm is nice to watch, but slows things down a bit
        #for idx in tqdm(list(range(len(leafNodes)))):
        for idx in range(len(leafNodes)):
            preState = leafNodes[idx]
            idPre  = stateD[preState] # listNodes.index(preState)
            state = preState.split(';')
            for move in moves_pool:
                move_direction = ['', '-']
                if ('globe' in puzzle_type) and ('f' in move): move_direction = ['']
                for reversed_move in move_direction:
                    move = reversed_move + move
                    next_state = move_state(state, move, moves_pool)
                    postState = ';'.join(next_state)
                    if postState not in stateD:
                        stateD[postState] = n # listNodes.append(postState)
                        #listNodes[n] = postState #.append(postState)
                        n += 1
                        newLeaves[k] = postState #.append(postState)
                        k += 1
                    idPost = stateD[postState] # listNodes.index(postState)
                    edgesD[idPre, idPost] = move
                    edgesR[idPost, idPre] = revMove(move)
            if (idx % 1000) == 0: memory_warning = check_memory()
            if memory_warning or (n > maxNodes) or (len(edgesD) > maxEdges):
                Done = True
                break
        if V: print(times_expanded,'   EXPANDED    nodes, leaves, edges', n, k, len(edgesD))
        leafNodes = np.copy(newLeaves[:k])
        
    print('Finished expanding neighbors.', times_expanded, 'iterations of expansion')
    print(len(stateD)/1000000, 'M nodes,', len(edgesD)/1000000, 'M edges. Elapsed', timedelta(seconds = (time.time() - ts)))
    
    g = ig.Graph(directed=False)
    g.add_vertices(n)
    g.add_edges(edgesD.keys())
    print('iGraph is now filled with all edges between nodes. Time', timedelta(seconds = (time.time() - ts)))
    # Call igraph to get shortest path through the graph
    listMovesNew = []
    shortestPath = g.get_shortest_paths(stateD[initialState], stateD[solutionState], output="vpath")[0]
    listMoves = []
    for j in range(len(shortestPath)-1):
        edge = (shortestPath[j], shortestPath[j+1])
        if edge in edgesD: listMoves.append(edgesD[edge])
        elif edge in edgesR: listMoves.append(edgesR[edge])
        else: print('PATH RECONSTRUCTION FAILS. edge not found')
    # **********************************
    # double check
    state = initialState.split(';')
    count = 0
    for move in listMoves:
        state = move_state(state, move, moves_pool)
        #print(count, move, ' state:', state)
        count+= 1
    
    # Check that submitted moves solve puzzle
    #num_wrong_facelets = beauty(solution_state, state)
    num_wrong_facelets = sum(not(s == t) for s, t in zip(solutionState, ';'.join(state)))
    print(' Total Elapsed', timedelta(seconds = (time.time() - ts)))
    if False:
        print('ending state:\n', ';'.join(state))
        print('\nsolution_state:\n', solutionState)
    #print('      count', count, '                  num_wrong_facelets', num_wrong_facelets)
    if num_wrong_facelets > num_wildcards:
        print('                  num_wrong_facelets', num_wrong_facelets)
        print('   *********************** UH OH ****************')
        print('   *********************** UH OH ****************')
        print('   *********************** UH OH ****************')
        print('   *********************** UH OH ****************')
        print('returning original solution for this puzzle becaue new solution fails!')
        print('ending state:\n', ';'.join(state))
        print('\nsolution_state:\n', solutionState)
        return('no joy!')
    else:
        print('    ------------------------------------------')
        print('    PUZZLE', puzzle_id, '    COST', count,    '( was', len(moves),' ) improvement', len(moves) - len(listMoves))
        print('    ------------------------------------------')
        if len(moves) <= len(listMoves):
            print('no joy')
            return('no joy!')
    return '.'.join(listMoves)

# Puzzles 337: Largest wreath type

In [8]:
ts = time.time()
gained = 0
MaxNodes = 60 * 1000 * 1000              
maxEdges = 412* 1000 * 1000

segment_length = 300                     
pids = [337] 

for i in pids:
    starting_score = len( submission_df.loc[i, 'moves'].split('.'))
    print('starting_score', starting_score)
    show_puzzle_info(i)
    for r in range(10):
        prior_score = len( submission_df.loc[i, 'moves'].split('.'))
        res = solve_random_puzzle_segment(i, MaxNodes, maxEdges, segment_length) # 
        #res = solve_entire_puzzle(i, MaxNodes, maxEdges,)
        submission_df.loc[i, 'moves'] = res
        submission_df.to_csv('submission_7.csv', index=False)
        resulting_score = len(res.split('.'))
        print(r, 'puzzle', i,'written as submission_7.csv, gained on puzzle', (starting_score - resulting_score), timedelta(seconds = (time.time() - ts)))
       
    gained += starting_score - resulting_score
    print('total gained so far', gained)
                
print('\n\n FINISHED NET IMPROVEMENT', gained)

starting_score 10518
puzzle_id 337        puzzle_type wreath_100/100
****** WILDCARDS ** 8 number of move types 2
number of moves in sample solution 10518
nunique_colors 3
number of faces 198
 state_list[] traced out. Elapsed 0:00:00.295220
start, end 8005 8305
clearing state_list[]
Memory is getting high: Your Jupyter notebook is using 25.001209259033203 GB of memory, which exceeds the safety limit of 25  GB.
We will wrap up work on this puzzle
Finished expanding neighbors. 12 iterations of expansion
26.613972 M nodes, 41.89412 M edges. Elapsed 0:27:43.370787
iGraph is now filled with all edges between nodes. Time 0:28:05.885034
 Total Elapsed 0:28:11.390838
    ------------------------------------------
    PUZZLE 337     COST 212 ( was 300  ) improvement 88
    ------------------------------------------
  spliced    count 10430                   num_wrong_facelets 8
0 puzzle 337 written as submission_7.csv, gained on puzzle 88 0:28:21.921767
 state_list[] traced out. Elapsed 0:00:00

# Puzzles 390,391

In [ ]:
ts = time.time()
MaxNodes = 60 * 1000 * 1000              
maxEdges = 412* 1000 * 1000

segment_length = 300                      
pids = [390,391] 

for i in pids:
    starting_score = len( submission_df.loc[i, 'moves'].split('.'))
    print('starting_score', starting_score)
    show_puzzle_info(i)
    for r in range(5):
        prior_score = len( submission_df.loc[i, 'moves'].split('.'))
        res = solve_random_puzzle_segment(i, MaxNodes, maxEdges, segment_length) # 
        #res = solve_entire_puzzle(i, MaxNodes, maxEdges,)
        submission_df.loc[i, 'moves'] = res
        submission_df.to_csv('submission_7.csv', index=False)
        resulting_score = len(res.split('.'))
        print(r, 'puzzle', i,'written as submission_7.csv, gained on puzzle', (starting_score - resulting_score), timedelta(seconds = (time.time() - ts)))
       
    gained += starting_score - resulting_score
    print('total gained so far', gained)
                
print('\n\n FINISHED NET IMPROVEMENT', gained)

starting_score 24741
puzzle_id 390        puzzle_type globe_3/33
number of move types 70
number of moves in sample solution 24741
nunique_colors 22
number of faces 264
 state_list[] traced out. Elapsed 0:00:00.840793
start, end 17355 17655
clearing state_list[]


# Puzzles 378

In [ ]:
ts = time.time()
#gained = 0 
MaxNodes = 60 * 1000 * 1000               
maxEdges = 400 * 1000 * 1000
segment_length = 150
pids = [378]

for i in pids:
    show_puzzle_info(i)
    starting_score = len( submission_df.loc[i, 'moves'].split('.'))
    print('starting_score', starting_score)
    for r in range(1):
        prior_score = len( submission_df.loc[i, 'moves'].split('.'))
        res = solve_entire_puzzle(i, MaxNodes, maxEdges,)
        submission_df.loc[i, 'moves'] = res
        submission_df.to_csv('submission_7.csv', index=False)
        resulting_score = len(res.split('.'))
        print(r, 'puzzle', i,'written as submission_7.csv, gained on puzzle', (starting_score - resulting_score), 'time', timedelta(seconds = (time.time() - ts)))
        if resulting_score >= prior_score - 3:
            print('Improvement slowing, moving on to another puzzle.')
            break
    gained += starting_score - resulting_score
    print(i,'total gained so far', gained)
                
print('\n\n FINISHED NET IMPROVEMENT', gained) 

# Puzzles 392, 396

In [ ]:
ts = time.time()
#gained = 0
MaxNodes = 60 * 1000 * 1000              
maxEdges = 412* 1000 * 1000
segment_length = 300                      
pids = [392,396] 
for i in pids:
    starting_score = len( submission_df.loc[i, 'moves'].split('.'))
    print('starting_score', starting_score)
    show_puzzle_info(i)
    for r in range(5):
        prior_score = len( submission_df.loc[i, 'moves'].split('.'))
        res = solve_random_puzzle_segment(i, MaxNodes, maxEdges, segment_length) # 
        #res = solve_entire_puzzle(i, MaxNodes, maxEdges,)
        submission_df.loc[i, 'moves'] = res
        submission_df.to_csv('submission_7.csv', index=False)
        resulting_score = len(res.split('.'))
        print(r, 'puzzle', i,'written as submission_7.csv, gained on puzzle', (starting_score - resulting_score), timedelta(seconds = (time.time() - ts)))
       
    gained += starting_score - resulting_score
    print('total gained so far', gained)
                
print('\n\n FINISHED NET IMPROVEMENT', gained)